## Let's try to compare the mmd of the 2 distributions to one nyström subsampled bucked for X and one not subsampled bucket for Y

In [26]:
from mmdew.bucket_stream2 import BucketStream
from mmdew.bucket_stream_old import BucketStream as OldBucketStream
import numpy as np
import pandas as pd
from mmdew.mmd import MMD
mymmd = MMD(biased=True, gamma=1)
from sklearn import metrics
import numpy.linalg as la
np.random.seed(40)
X = np.random.normal(0, 1, 32)
Y = np.random.normal(1, 2, 32)


m_idx = np.random.default_rng().integers(32, size=5)
subsample = X[m_idx]
start_weights = ((1/32) * la.pinv(metrics.pairwise.linear_kernel(subsample.reshape(-1,1),subsample.reshape(-1,1)))
                 @ metrics.pairwise.linear_kernel(subsample.reshape(-1,1),X.reshape(-1,1)) @ np.ones(32))



end_weights = np.ones((32,1))/32
addend_1 = start_weights.T @ metrics.pairwise.linear_kernel(subsample.reshape(-1,1), subsample.reshape(-1,1)) @ start_weights
addend_2 = end_weights.T @ metrics.pairwise.linear_kernel(Y.reshape(-1,1), Y.reshape(-1,1)) @ end_weights
addend_3 = start_weights.T @ metrics.pairwise.linear_kernel(subsample.reshape(-1,1), Y.reshape(-1,1)) @ end_weights
print(f"real mmd:{mymmd.mmd(X.reshape(-1,1),Y.reshape(-1,1))}")
print(f"nyström mmd: {(addend_1 + addend_2 - (2 * addend_3))[0][0]}")



real mmd:0.6016100063216595
nyström mmd: 0.6016100063216596


## MMD is identical
now let's try to create 2 subsampled buckets from X, merge them into a third bucket with the elements: new_subsample and the weights: new_weights, then check MMD

In [27]:
import math

X_1 = np.random.normal(0, 1, 32)
X_2 = np.random.normal(0, 1, 32)
Y = np.random.normal(1, 2, 64)


m_idx1 = np.random.default_rng().integers(32, size=5)
subsample1 = X_1[m_idx1]
m_idx2 = np.random.default_rng().integers(32, size=5)
subsample2 = X_2[m_idx1]
bucket1_weights = ((1/32) * la.pinv(metrics.pairwise.linear_kernel(subsample1.reshape(-1,1),subsample1.reshape(-1,1)))
                 @ metrics.pairwise.linear_kernel(subsample1.reshape(-1,1),X_1.reshape(-1,1)) @ np.ones(32))
bucket2_weights = ((1/32) * la.pinv(metrics.pairwise.linear_kernel(subsample2.reshape(-1,1),subsample2.reshape(-1,1)))
                 @ metrics.pairwise.linear_kernel(subsample2.reshape(-1,1),X_2.reshape(-1,1)) @ np.ones(32))

#getting new elements
joined_subsamples = np.concatenate((subsample1, subsample2))
joined_uncompressed_capacity = 64
m = round(math.sqrt(64))
m_idx_new = np.random.default_rng().integers(len(joined_subsamples), size=m)
new_subsample = joined_subsamples[m_idx_new]

#getting new weights
new_subsample = new_subsample.reshape(-1,1)
joined_subsamples = joined_subsamples.reshape(-1,1)
joined_weights = np.concatenate((bucket1_weights,bucket2_weights))
K_z = metrics.pairwise.linear_kernel(new_subsample, joined_subsamples)
K_m_inv = la.pinv(metrics.pairwise.linear_kernel(new_subsample, new_subsample))
new_weights = .5 * K_m_inv @ K_z @ joined_weights

#weights for Y
end_weights = np.ones((64,1))/64

addend_1 = new_weights.T @ metrics.pairwise.linear_kernel(new_subsample.reshape(-1,1), new_subsample.reshape(-1,1)) @ new_weights
addend_2 = end_weights.T @ metrics.pairwise.linear_kernel(Y.reshape(-1,1), Y.reshape(-1,1)) @ end_weights
addend_3 = new_weights.T @ metrics.pairwise.linear_kernel(new_subsample.reshape(-1,1), Y.reshape(-1,1)) @ end_weights

print(f"real mmd:{mymmd.mmd(np.concatenate((X_1,X_2)).reshape(-1,1),Y.reshape(-1,1))}")
print(f"nyström mmd of bucket, made of subsampled buckets: {(addend_1 + addend_2 - (2 * addend_3))[0][0]}")
breakpoint()




real mmd:2.3226094951499947
nyström mmd of bucket, made of subsampled buckets: 2.322609495149995


## MMD is equal 
why does this not work in bucketstream2.py ? 

In [28]:
from mmdew.bucket_stream2 import BucketStream
from mmdew.bucket_stream_old import BucketStream as OldBucketStream
import numpy as np
import pandas as pd
from mmdew.mmd import MMD
mymmd = MMD(biased=True, gamma=1)
if __name__ == "__main__":
    bs_ss = BucketStream(gamma=1)
    bs_no_ss = BucketStream(gamma=1, apply_subsampling=False)
    bs_old = OldBucketStream(gamma=1)

    errors = []

    for exponent in range(1, 11):
        iter_errors = []
        for iteration in range(1,11):

            bs_ss = BucketStream(gamma=1)
            bs_no_ss = BucketStream(gamma=1, apply_subsampling=False)
            bs_old = OldBucketStream(gamma=1)
            limit = 2 ** exponent
            X = np.random.normal(0, 1, limit)
            Y = np.random.normal(1, 2, limit - 1)
           
            for i in range(0, limit):
                bs_ss.insert_no_cut(X[i])
             
            for i in range(0, limit - 1):
                bs_ss.insert_no_cut(Y[i])
      
            ss_mmd = bs_ss.mmd(1)[0]
            real_mmd = mymmd.mmd(X.reshape(-1, 1),Y.reshape(-1, 1))
            #print(f"SS:    exponent: {exponent} bucket count: {len(bs_ss.buckets)} mmd value={ss_mmd}")
         
            #print(f"real MMD: {real_mmd}")
            iter_errors = np.concatenate(( iter_errors, [(abs(real_mmd - ss_mmd))]))
            #if iteration == 1 and exponent == 5 :
            #    breakpoint()
        errors = np.concatenate((errors, [np.array(iter_errors).mean()] ))

    for i in range(0,10):
        print(f"error for exponent {i+1} is {errors[i]}")


error for exponent 1 is 6.938893903907228e-16
error for exponent 2 is 1.2212453270876722e-15
error for exponent 3 is 1.2212453270876722e-15
error for exponent 4 is 6.161737786669619e-16
error for exponent 5 is 7.327471962526033e-16
error for exponent 6 is 0.27322697803554885
error for exponent 7 is 0.29218514919762295
error for exponent 8 is 0.2324963532574406
error for exponent 9 is 0.24656608561197074
error for exponent 10 is 0.1989622980445256


# Exponent 6 is the first time that we subsample with depth > 1 
But we just showed that this isn't the problem. Let's have a deeper look

In [47]:
bs_ss = BucketStream(gamma=1)
exponent = 6
limit = 2 ** exponent
X = np.random.normal(0, 1, limit)
Y = np.random.normal(1, 2, limit - 1)

for i in range(0, limit):
    bs_ss.insert_no_cut(X[i])
 
for i in range(0, limit - 1):
    bs_ss.insert_no_cut(Y[i])

ss_mmd = bs_ss.mmd(1)[0]
real_mmd = mymmd.mmd(X.reshape(-1, 1),Y.reshape(-1, 1))
print(f"SS:    exponent: {exponent} bucket count: {len(bs_ss.buckets)} mmd value={ss_mmd}")
print(f"real MMD: {real_mmd}\n")
for bucket in bs_ss.buckets:
    print(f"Element count: {len(bucket.elements)}" )
    print(bucket)
    print("\n")



SS:    exponent: 6 bucket count: 7 mmd value=0.6252552407740909
real MMD: 0.44729737365887745

Element count: 8
Elems:	[[ 0.18417749]
 [ 0.18417749]
 [ 0.4931863 ]
 [ 0.06152661]
 [-1.47501882]
 [ 0.06152661]
 [ 0.18417749]
 [ 0.06152661]]
Weights:	[[ 0.00922894]
 [ 0.00922894]
 [ 0.02471303]
 [ 0.00308303]
 [-0.07391161]
 [ 0.00308303]
 [ 0.00922894]
 [ 0.00308303]]
Uncomp_Cap:	64


Element count: 6
Elems:	[[ 1.08304634]
 [-1.48046468]
 [-2.17882162]
 [ 0.77960253]
 [-0.45225296]
 [-0.45225296]]
Weights:	[[ 0.07381375]
 [-0.10089933]
 [-0.14849502]
 [ 0.05313289]
 [-0.03082277]
 [-0.03082277]]
Uncomp_Cap:	32


Element count: 16
Elems:	[[ 2.08629853]
 [ 2.44841171]
 [ 3.64384188]
 [-0.18678735]
 [-3.48382213]
 [ 0.0660403 ]
 [-1.05167316]
 [ 0.41578454]
 [-2.10610054]
 [ 2.44495657]
 [ 2.34850021]
 [ 1.99299365]
 [-0.08587857]
 [ 2.26389168]
 [ 2.44022949]
 [ 0.24652004]]
Weights:	[[ 0.02580577]
 [ 0.03028481]
 [ 0.04507128]
 [-0.0023104 ]
 [-0.04309197]
 [ 0.00081686]
 [-0.01300832]
 

## This does not look to bad. (Except for the mmd disparity of course)
The amount of subsamples is right. the weight count matches...
Let's try to reconstruct the mmd calc

In [48]:
start = bs_ss.buckets[:1]
end = bs_ss.buckets[1:]
start_elements = start[0].elements
start_weights = start[0].weights * len(start[0].weights)
end_elements = end[0].elements
end_weights = end[0].weights * len(end[0].weights)
start_uncompressed_capacity = start[0].uncompressed_capacity
end_uncompressed_capacity = end[0].uncompressed_capacity
for bucket in start[1:]:
    #breakpoint()
    start_elements = np.concatenate((start_elements, bucket.elements))
    start_weights = np.concatenate((start_weights, bucket.weights * len(bucket.weights)))
    start_uncompressed_capacity += bucket.uncompressed_capacity
for bucket in end[1:]:
    #breakpoint()
    end_elements = np.concatenate((end_elements, bucket.elements))
    #breakpoint()
    end_weights = np.concatenate((end_weights, bucket.weights * len(bucket.weights)))
    end_uncompressed_capacity += bucket.uncompressed_capacity
start_capacity = len(start_elements)
end_capacity = len(end_elements)
print(start_uncompressed_capacity)
print(start_capacity)
print(end_uncompressed_capacity)
print(end_capacity)

64
8
63
37


## Yes, everything seems to be alright
37 is exactly the amount of samples that should be right of the split because of: 31 elements from the buckets without subsampling and 5 (+1 for bad rounding) from the subsampled u-cap 32 bucket
## Let's look deeper...

In [49]:
start_weights = start_weights * (1/start_capacity)
end_weights = end_weights * (1/end_capacity)
#breakpoint()
print("Start Weights")
print(start_weights)
print("\nEnd Weights")
print(end_weights)

Start Weights
[[ 0.00922894]
 [ 0.00922894]
 [ 0.02471303]
 [ 0.00308303]
 [-0.07391161]
 [ 0.00308303]
 [ 0.00922894]
 [ 0.00308303]]

End Weights
[[ 0.0119698 ]
 [-0.01636205]
 [-0.02408027]
 [ 0.00861614]
 [-0.00499829]
 [-0.00499829]
 [ 0.01115925]
 [ 0.01309613]
 [ 0.01949028]
 [-0.00099909]
 [-0.01863437]
 [ 0.00035324]
 [-0.00562522]
 [ 0.00222396]
 [-0.01126517]
 [ 0.01307765]
 [ 0.01256172]
 [ 0.01066018]
 [-0.00045935]
 [ 0.01210917]
 [ 0.01305237]
 [ 0.00131859]
 [ 0.01376214]
 [ 0.02001749]
 [-0.00188383]
 [ 0.00689044]
 [ 0.01929372]
 [-0.01451079]
 [-0.0039672 ]
 [ 0.0098713 ]
 [ 0.03664056]
 [ 0.02023039]
 [ 0.01134437]
 [ 0.02558738]
 [-0.00191896]
 [ 0.024786  ]
 [ 0.02702703]]


In [50]:
addend_1 = start_weights.T @ metrics.pairwise.linear_kernel(start_elements, start_elements) @ start_weights
addend_2 = end_weights.T @ metrics.pairwise.linear_kernel(end_elements, end_elements) @ end_weights
addend_3 = start_weights.T @ metrics.pairwise.linear_kernel(start_elements, end_elements) @ end_weights
print(addend_1)
print(addend_2)
print(addend_3)
print(addend_1+addend_2-(2*addend_3))

[[0.0160979]]
[[0.84200502]]
[[0.11642384]]
[[0.62525524]]
